In [ ]:
import cv2
import numpy as np
from dataset_maker import datasetmaker,data_browser
import torch

In [ ]:
datamaker  = datasetmaker("../sp_data/train.mp4")
points = datamaker.get_points_video()

In [ ]:
#these would be final as whole model depends upon it. As width and height calculated depends on these points
points

In [ ]:
count = 0
for tens in datamaker.generate_tensor_data(points):
    print(tens)
    count+=1
    if(count==5):
        break

In [ ]:
tens.shape